In [2]:
import pandas as pd
import time 
#import ast
from all_functions import *
import math
import requests
from pylatex import Document, Section, Subsection, Command
from pylatex.utils import italic, NoEscape
from selenium import webdriver
import os
import subprocess
import urllib.request
import codecs
import unidecode
import sys

In [3]:
def get_address(query, api_key):
        base = 'https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input='
        locationbias= '42.3314584645106, -71.1039191294055'
        #key = api_key
        query = query
        fields = 'formatted_address,name,geometry'
        inputtype = 'textquery'
        
        url_search = base + query + '&inputtype=' + inputtype + '&fields=' + fields + '&locationbias=circle:2000@' + locationbias + '&key=' + api_key
        print(url_search)
        my_dict = requests.get(url_search).json()
        return my_dict

In [4]:
def get_commute(dest, origin, api_key):
    
    base = 'https://maps.googleapis.com/maps/api/distancematrix/json?units=imperial&'
    origin = 'origins=' + origin + '&'
    dest_str = 'destinations=' + dest + '&'
    api_key = 'key=' + api_key
    
    url_search = base + origin + dest_str + api_key
    print(url_search)
    my_dict = requests.get(url_search).json()
    return my_dict

In [5]:
def dist_duration(company_list, id_list, api_key):
    comm_df = pd.DataFrame(columns=['company', 'place', 'address', 'distance', 'duration'])
    for index, company in enumerate(company_list):
        comm_dict = {}
        company = company
        tmp_id = id_list[int(index)]
        text_dict = get_address(str(company), api_key=api_key)
        #print(text_dict)
        if text_dict['status'] == 'ZERO_RESULTS':
            print(company + ' has 0 results in get_address.')
            base = 'https://maps.googleapis.com/maps/api/staticmap?'
            place = 'None'
            address = 'None'
            distance = 'None'
            duration = 'None'
            #params = {'center' : '8 Oswald Street, Boston MA 02120' , 'zoom' : '20', 'size' : '200x200' , 'maptype' : 'roadmap', 'key': api_key}
            params = {'center' : '8 Oswald Street, Boston MA 02120' , 'zoom' : '10', 'size' : '200x200' , 'maptype' : 'roadmap', 'key': api_key}
            map_url = requests.get(base, params).url
        else:
            address = text_dict['candidates'][0]['formatted_address']
            place = text_dict['candidates'][0]['name']    
            res_lat_lng = str(text_dict['candidates'][0]['geometry']['location']['lat']) + ',' + str(text_dict['candidates'][0]['geometry']['location']['lng'])
            base = 'https://maps.googleapis.com/maps/api/staticmap?'
            #params = {'center' : '8 Oswald Street, Boston MA 02120' , 'zoom' : '18', 'size' : '200x200' , 'maptype' : 'roadmap', 'markers': 'color:blue|label:S|' + res_lat_lng, 'key': api_key} 
            params = {'center' : '8 Oswald Street, Boston MA 02120' , 'size' : '200x200' , 'maptype' : 'roadmap', 'markers': 'color:blue|label:S|' + res_lat_lng, 'key': api_key} 
            map_url = requests.get(base, params).url
            
            
            comm = get_commute(dest=res_lat_lng, origin='8 Oswald Street, 02120', api_key=api_key)
            if comm['rows'][0]['elements'][0]['status'] == 'ZERO_RESULTS':
                print(company + ' has 0 results in get_commute.')
                continue
            else:
                try:
                    distance = (comm['rows'][0]['elements'][0]['distance']['text'])
                    duration = (comm['rows'][0]['elements'][0]['duration']['text'])
                except:
                    #print(comm)
                    break

        comm_dict['company'] = company
        comm_dict['place'] = place
        comm_dict['address'] = address
        comm_dict['distance'] = distance
        comm_dict['duration'] = duration
        comm_dict['ID'] = tmp_id
        comm_dict['map_url'] = map_url
        comm_tmp_df = pd.DataFrame(comm_dict, index = [comm_dict['ID']])
        comm_df = comm_df.append(comm_tmp_df, sort=False)
    return comm_df

In [6]:
def stack_scraper(wd, results, job_title, city):
    
    res_left = results
    tot_pg = ((results // 25) + 1)
    print('stack total pages: ' + str(tot_pg))
    #print('indeed total pages: ' + str(tot_pg))
    ID = [str('s-ID-' + str(n)) for n in range(0, results)]
    pulldates = [str(time.strftime("%m%d%Y", time.localtime(time.time()))) for n in range(0, results)]
    stack_df = pd.DataFrame({'ID' : ID, 'pulldates' : pulldates}).set_index('ID')
    for i in range(tot_pg):
        url = 'https://stackoverflow.com/jobs?q=python&sort=p&l=boston&d=20&u=Miles&start=' + str(i*25)
        pg_IDs = ID[i*25:(i+1)*25]
        post_links = [title.get_attribute('href') for title in wd.find_elements_by_xpath("//h2[@class='fs-subheading job-details__spaced mb4']//a")]
        #url = 'https://www.indeed.com/jobs?q=python&l=worcester&start=0&sort=date&radius=25'
        #print(url)
        wd.get(url)
        #################### search results ####################     
        #comp_locs = [comp_loc.text for comp_loc in wd.find_elements_by_xpath("//div[@class='-job-summary']/div[contains(@class, '-company')]")]
        stack_df.loc[pg_IDs, 'comp_locs'] = [comp_loc.text for comp_loc in wd.find_elements_by_xpath("//div[@class='-job-summary']/div[contains(@class, '-company')]")]
        test_len = len([comp_loc.text for comp_loc in wd.find_elements_by_xpath("//div[@class='-job-summary']/div[contains(@class, '-company')]")])
        
        res_len = min(test_len, int(res_left))

        #l_companies = [company.text.lower().split(',')[0] for company in wd.find_elements_by_xpath("//div[@data-tn-component='organicJob']//span[@class='company']")]
        stack_df.loc[pg_IDs, 'titles'] = [title.text for title in wd.find_elements_by_xpath("//h2[@class='fs-subheading job-details__spaced mb4']")]
        stack_df.loc[pg_IDs, 'comp_links'] = ['https://www.stackoverflow.com/jobs/company/' + company.text.replace(' ', '-') for company in wd.find_elements_by_xpath("//div[@class='fc-black-700 fs-body2 -company']")]
        stack_df.loc[pg_IDs, 'tags'] = [tag.text for tag in wd.find_elements_by_xpath("//div[@class='mt12 -tags']")]
        #stack_df.loc[pg_IDs, 'perks'] = [perk.text for perk in wd.find_elements_by_xpath("//div[@class='mt2 -perks']")]
        stack_df.loc[pg_IDs, 'postdates'] = [date.text for date in wd.find_elements_by_xpath("//span[contains(@class, 'ps-absolute pt2 r0 fc-black-500 fs-body1 pr12')]")]
        #description = [description.text for description in wd.find_elements_by_xpath("//div[@data-tn-component='organicJob']//span[@class='summary']")]
        stack_df.loc[pg_IDs, 'post_links'] = [title.get_attribute('href') for title in wd.find_elements_by_xpath("//h2[@class='fs-subheading job-details__spaced mb4']//a")]

        #comp_links = ['https://www.indeed.com/cmp/' + str(company).replace(' ', '-') for company in companies]                    

        '''        if res_len < 25:
            comp_locs = comp_locs[:res_len]
            titles = titles[:res_len]
            tags = tags[:res_len]
            perks = perks[:res_len]
            post_links = post_links[:res_len]
            postdates = postdates[:res_len]
            comp_links = comp_links[:res_len]'''
            
            
        #ID = [str('s-ID-' + str(n)) for n in range(results-res_left, results-res_left+len(comp_locs))]
        #comp_loc = ['' for n in range(results-res_left, results-res_left+len(comp_locs))]
        #companies = [company.split(' - ')[0] for company in comp_locs]
        #locations = [location.split(' - ')[1] for location in comp_locs]
                    
        res_left = res_left - res_len

        #print(len(titles), len(companies), len(post_links), len(postdates), len(pulldates), len(comp_links))
        #return 'bitch'
        #stack_df = pd.DataFrame({'ID' : ID, 'title' : titles, 'company' : companies, 'post_link' : post_links, 'comp_link' : comp_links, 'postdate' : postdates, 'pulldate' : pulldates}).set_index('ID') # functional

        
        #################### post list ####################     
        post_list = []
        for index, link in enumerate(post_links):
            post_id = ID[index]
            wd.get(link)
            #average_comp_rating = wd.find_element_by_xpath("//span[@class='cmp-header-rating-average']").text
            stack_df.loc[post_id, 'body'] = wd.find_element_by_xpath("//div[@id='overview-items']").text
            try:
                stack_df.loc[post_id, 'benefits'] = wd.find_element_by_xpath("//section[@class='-benefits mb32']").text
            except:
                stack_df.loc[post_id, 'benefits'] = ''
                #print(link)
            res_len = res_len - len(post_links)    
            
            #post_list.append([str(index), body, benefits])
        #comp_df = pd.DataFrame(post_list, columns=['ID', 'body_text', 'benefits']).set_index('ID')
        #stack_df = stack_df.merge(comp_df)
    return stack_df

In [ ]:
def linkedin_scraper(wd, results, job_title, city, linkedin_username, linkedin_password):
    tot_pg = ((results // 25) + 1)
    res_left = results
    print('linkedin total pages: ' + str(tot_pg))
    login_url = 'https://www.linkedin.com/'
    wd.get(login_url)
    #wd.find_element_by_xpath("//a[@class='sign-in-link']").click
    #wd.find_element_by_xpath("//a[@class='form-toggle']").click
    login_email = wd.find_element_by_id("login-email")
    login_email.send_keys(linkedin_username)
    login_pass = wd.find_element_by_id("login-password")
    login_pass.send_keys(linkedin_password)
    wd.find_element_by_id("login-submit").click
    #cookies = wd.get_cookie('bcookie')
    
    ID = [('L-ID-' + str(n)) for n in range(0, results)]
    pulldates = [str(time.strftime("%m%d%Y", time.localtime(time.time()))) for n in range(0, results)]
    linkedin_df = pd.DataFrame({'ID' : ID, 'pulldates' : pulldates}).set_index('ID')
    for i in range(tot_pg):
        url = 'http://www.linkedin.com/jobs/search/?keywords=' + job_title + '&location=' + city + '&sortBy=DD&start=' + str(i*25)
        print('Page {} url : {}'.format(i, url))
        wd.get(url)
        pg_IDs = ID[i*25:(i+1)*25] # Gets 25 IDs relevant to page
        res_len = min(len(wd.find_elements_by_xpath("//li[@class='job-listing']")), int(res_left))
        companies = [c.text for c in wd.find_elements_by_xpath("//span[@class='company-name-text']")]
        locations = [L.text for L in wd.find_elements_by_xpath("//span[@class='job-location']/span")]
        comp_links = []
        for elem in wd.find_elements_by_xpath("//div[@class='company-name']"):
            if elem.find_element_by_xpath("//a") == None:
                comp_links.append('NA')
            else:
                comp_links.append(elem.find_element_by_xpath("//a").get_attribute('href'))
        linkedin_df.loc[pg_IDs, 'comp_links'] = comp_links
        linkedin_df.loc[pg_IDs, 'post_links'] = [l.get_attribute('href') for l in wd.find_elements_by_xpath("//a[@class='job-title-link']")]
        linkedin_df.loc[pg_IDs, 'titles'] = [t.text for t in wd.find_elements_by_xpath("//span[@class='job-title-text']")]
        linkedin_df.loc[pg_IDs, 'locations'] = [L.text for L in wd.find_elements_by_xpath("//span[@class='job-location']/span")]
        linkedin_df.loc[pg_IDs, 'companies'] = [c.text for c in wd.find_elements_by_xpath("//span[@class='company-name-text']")]
        linkedin_df.loc[pg_IDs, 'descr'] = [d.text for d in wd.find_elements_by_xpath("//div[@class='job-description']")]
        linkedin_df.loc[pg_IDs, 'ezapply'] = [e.text for e in wd.find_elements_by_xpath("//div[@class='job-flavor-in-apply-container']")]
        linkedin_df.loc[pg_IDs, 'company_pics'] = [i.get_attribute('src') for i in wd.find_elements_by_xpath("//a[@class='company-logo-link']//img")]

        comp_loc = []
        for i, elem in enumerate(companies):
            comp_loc_str = (elem.strip().split(',')[1:] + ', ' + locations[i].strip())
            comp_loc.append(comp_loc_str)
        linkedin_df.loc[pg_IDs, 'comp_loc'] = comp_loc
        res_left = res_left - res_len
    return linkedin_df

In [45]:
def indeed_scraper(wd, results, job_title, city):
    list_o_divs = []
    #tot_pg = ((results // 10) + 1)
    res_left = results
    tot_pg = ((results // 10) + 1)
    print('indeed total pages: ' + str(tot_pg))
    #print('indeed total pages: ' + str(tot_pg))
    ID = [('i-ID-' + str(n)) for n in range(0, results)]
    pulldates = [str(time.strftime("%m%d%Y", time.localtime(time.time()))) for n in range(0, results)]
    indeed_df = pd.DataFrame({'ID' : ID, 'pulldate' : pulldates}).set_index('ID')
    for i in range(tot_pg):
        url = 'https://www.indeed.com/jobs?q=' + job_title + '&l=' + city + '&start=' + str(i*10) + '&sort=' + 'date' + '&radius=' + '25'
        pg_IDs = ID[i*10:(i+1)*10] # Gets 25 IDs relevant to page
        #url = 'https://www.indeed.com/jobs?q=python&l=worcester&start=0&sort=date&radius=25'
        print('Page {} url : {}'.format(i, url))
        wd.get(url)
        #################### search results ####################     
        
        listing_len = len([job for job in wd.find_elements_by_xpath("//div[@data-tn-component='organicJob']")])
        res_len = min(listing_len, int(res_left))
        
        indeed_df.loc[pg_IDs, 'companies'] = [company.text for company in wd.find_elements_by_xpath("//div[@data-tn-component='organicJob']//span[@class='company']")][:res_len]

        #res_len = min(listing_len, int(res_left))

        indeed_df.loc[pg_IDs, 'l_companies'] = [company.text.lower().split(',')[0] for company in wd.find_elements_by_xpath("//div[@data-tn-component='organicJob']//span[@class='company']")][:res_len]
        indeed_df.loc[pg_IDs, 'titles'] = [title.text for title in wd.find_elements_by_xpath("//div[@data-tn-component='organicJob']//h2[@class='jobtitle']")][:res_len]
        indeed_df.loc[pg_IDs, 'description'] = [description.text for description in wd.find_elements_by_xpath("//div[@data-tn-component='organicJob']//span[@class='summary']")][:res_len]
        indeed_df.loc[pg_IDs, 'post_links'] = [post_link.get_attribute('href') for post_link in wd.find_elements_by_xpath("//div[@data-tn-component='organicJob']//h2[@class='jobtitle']//a")][:res_len]
        indeed_df.loc[pg_IDs, 'comp_links'] = ['https://www.indeed.com/cmp/' + str(company.text).replace(' ', '-') for company in wd.find_elements_by_xpath("//div[@data-tn-component='organicJob']//span[@class='company']")][:res_len]           
        post_links = [post_link.get_attribute('href') for post_link in wd.find_elements_by_xpath("//div[@data-tn-component='organicJob']//h2[@class='jobtitle']//a")][:res_len]
        comp_links = ['https://www.indeed.com/cmp/' + str(company).replace(' ', '-') for company in wd.find_elements_by_xpath("//div[@data-tn-component='organicJob']//span[@class='company']")][:res_len]
        '''if res_len < 25:
            l_companies = l_companies[:res_len]
            titles = titles[:res_len]
            description = description[:res_len]
            companies = companies[:res_len]
            post_links = post_links[:res_len]
            comp_links = comp_links[:res_len]'''

        

        #list_df = pd.DataFrame({'ID' : ID, 'title' : titles, 'company' : companies, 'summary' : description, 'post_link' : post_links, 'comp_link' : comp_links, 'l_company' : l_companies, 'pulldate' : pulldates}) # functional
        #list_df['ID'] = list_df['ID'].astype('str')

        #################### comp list ####################     
        comp_list = []
        for index, link in enumerate(comp_links):
            index = ID[index]
            wd.get(link)
            try:
                indeed_df.loc[index, 'company_name'] = wd.find_element_by_xpath("//div[@class='cmp-company-name-container']").text
                indeed_df.loc[index, 'l_company_name'] = wd.find_element_by_xpath("//div[@class='cmp-company-name-container']").text.lower().split(',')[0]
                indeed_df.loc[index, 'salaries_link'] = 'https://www.indeed.com/cmp/' + wd.find_element_by_xpath("//div[@class='cmp-company-name-container']").text + '/salaries/'
                indeed_df.loc[index, 'reviews_link'] = 'https://www.indeed.com/cmp/' + wd.find_element_by_xpath("//div[@class='cmp-company-name-container']").text + '/reviews/'
                try:
                    indeed_df.loc[index, 'average_comp_rating'] = wd.find_element_by_xpath("//span[@class='cmp-header-rating-average']").text
                except:
                    indeed_df.loc[index, 'average_comp_rating'] = 'Not ranked.'
            except:
                indeed_df.loc[index, 'company_name'] = ''
                indeed_df.loc[index, 'l_company_name'] = ''
                indeed_df.loc[index, 'salaries_link'] = ''
                indeed_df.loc[index, 'reviews_link'] = ''
                continue
            
            #comp_list.append([str(index), company_name, average_comp_rating, salaries_link, reviews_link, l_company_name])
        #comp_df = pd.DataFrame(comp_list, columns=['ID', 'company', 'rating', 'salaries_link', 'reviews_link', 'l_company'])

        #################### Post list ####################     
        post_list = []
        for index, link in enumerate(post_links):
            index = ID[index]
            wd.get(link)
            indeed_df.loc[index, 'jobtitle'] = wd.find_element_by_xpath("//h3[contains(@class, 'jobsearch-JobInfoHeader-title')]").text
            try:
                indeed_df.loc[index, 'company_pic'] = wd.find_element_by_xpath("//img[@class='jobsearch-CompanyAvatar-image']").get_attribute('src')
            except:
                indeed_df.loc[index, 'company_pic'] = 'NA'
            line = wd.find_element_by_xpath("//div[contains(@class, 'jobsearch-InlineCompanyRating')]").text
            indeed_df.loc[index, 'location'] = line.split('\n')[-1]
            indeed_df.loc[index, 'company_name'] = line.split('\n')[0]
            indeed_df.loc[index, 'l_company_name'] = line.split('\n')[0].lower().split(',')[0]
            indeed_df.loc[index, 'comp_loc'] = line.split('\n')[0] + ', ' + line.split('\n')[-1]
            indeed_df.loc[index, 'body'] = wd.find_element_by_xpath("//div[contains(@class, 'jobsearch-JobComponent-description')]").text
            indeed_df.loc[index, 'post_time'] = wd.find_element_by_xpath("//div[@class='jobsearch-JobMetadataFooter']").text.split(' - ')[1].strip()
            

        #post_df = pd.DataFrame(post_list, columns=['ID', 'title', 'company', 'comp_loc', 'postdate', 'location', 'company_pic', 'body_text', 'l_company'])

        #################### DF merging ####################                    
        #post_comp_df = comp_df.merge(post_df, on=['ID'], how='right').drop(columns=['l_company_y', 'company_y']).rename({'company_x' : 'company', 'l_company_x' : 'l_company'}, axis='columns')
        #post_comp_df['ID'] = post_comp_df['ID'].astype('str')
        #tmp_df = post_comp_df.merge(list_df, on=['ID'], how='outer').drop(columns=['l_company_y', 'company_y', 'title_y']).rename({'company_x' : 'company', 'l_company_x' : 'l_company', 'title_x' : 'title'}, axis='columns')
        #indeed_df = indeed_df.append(tmp_df, sort=False)

        #num += len(tmp_df)
        res_left = res_left - res_len
    return indeed_df

In [49]:
def ult_scrap(**scrap_params):
    results = scrap_params['results_per_site']
    job_title = scrap_params['job_title']
    city = scrap_params['city'] 
    site_list = scrap_params['site_list']
    linkedin_username = scrap_params['linked_in_username'] 
    linkedin_password = scrap_params['linked_in_password']
    chromedriver_location = scrap_params['chromedriver_location'] 
    api_key = scrap_params['api_key'] 
    
    file = 1
    SKIPPER = 0 # Skips this many of the first jobs
    #job_col_list = ['site', 'sID',  'title', 'company', 'location', 'salary', 'summary', 'tags','id', 'comp_loc', 'link', 'pulldate','postdate', 'ezapply']
    #job_df = pd.DataFrame(columns=job_col_list).set_index('sID')
    #indeed_df = pd.DataFrame()
    #linkedin_df = pd.DataFrame()
    #stack_df = pd.DataFrame()

    num = 0
    options = Options()
    for arg in ['--headless', '--no-sandbox', '--disable-gpu', 'start-maximized', 'disable-infobars', '--disable-extensions']:
        options.add_argument(arg)
    '''options.add_argument("--headless") # Runs Chrome in headless mode.
    options.add_argument('--no-sandbox') # Bypass OS security model
    options.add_argument('--disable-gpu')  # applicable to windows os only
    options.add_argument('start-maximized') # 
    options.add_argument('disable-infobars')
    options.add_argument("--disable-extensions")'''
    wd = webdriver.Chrome(chromedriver_location, options=options)
    job_df = pd.DataFrame()
    for site in site_list: 
        site_ind = results
        if site == 'stack':
            stack_df = stack_scraper(wd, results, job_title, city)
            job_df = job_df.append(stack_df, sort=False)
            
        if site == 'indeed':
            indeed_df = indeed_scraper(wd, results, job_title, city)
            job_df = job_df.append(indeed_df, sort=False)
                    
        if site == 'linkedin':
            linkedin_df = linkedin_scraper(wd, results, job_title, city, linkedin_username, linkedin_password)
            job_df = job_df.append(linkedin_df, sort=False)
    return job_df
    #job_df = linkedin_df.append(indeed_df, sort=False)
    #job_df = job_df.append(stack_df, sort=False)
    try:
        job_df.set_index('ID', inplace=True)
    except:
        print('fail')
        return indeed_df
    companies = job_df['comp_loc']
    ids = job_df.index.tolist()
    
    
    
    comm_df = dist_duration(companies, ids, api_key)

    fin_df = job_df.merge(comm_df, how = 'left', left_index=True, right_on=['ID']).drop(columns=['company_x']).rename({'company_y' : 'company'}, axis='columns')

    file = file + 1
    wd.stop_client()
    wd.quit()
    output_dict = {'job_title' : job_title, 'city' : city}
    return fin_df


In [51]:
fetch = ult_scrap(**{
    'results_per_site' : 25,
    'job_title' : 'python',
    'city' : 'boston',
    'site_list' : ['indeed', 'linkedin'],
    'linked_in_username' : 'USERNAME',
    'linked_in_password' : 'PASSWORD',
    'chromedriver_location' : u'/chromedriver.exe',
    'api_key' : 'APIKEY'
})
fetch

indeed total pages: 3
Page 0 url : https://www.indeed.com/jobs?q=python&l=boston&start=0&sort=date&radius=25
Page 1 url : https://www.indeed.com/jobs?q=python&l=boston&start=10&sort=date&radius=25
Page 2 url : https://www.indeed.com/jobs?q=python&l=boston&start=20&sort=date&radius=25
linkedin total pages: 2
Page 0 url : http://www.linkedin.com/jobs/search/?keywords=python&location=boston&sortBy=DD&start=0
Page 1 url : http://www.linkedin.com/jobs/search/?keywords=python&location=boston&sortBy=DD&start=25


,pulldate,companies,l_companies,titles,description,post_links,comp_links,company_name,l_company_name,salaries_link,...,company_pic,location,comp_loc,body,post_time,pulldates,locations,descr,ezapply,company_pics
ID,,,,,,,,,,,,,,,,,,,,,
i-ID-0,11162018,Ace Technologies,ace technologies,QA Analyst,This person will be doing automation testing o...,https://www.indeed.com/rc/clk?jk=827870af9af1e...,https://www.indeed.com/cmp/Ace-Technologies,IBM,ibm,,...,https://d2q79iu7y748jz.cloudfront.net/s/_logo/...,"Cambridge, MA 02139","IBM, Cambridge, MA 02139",Job Description\nIBM Research is the innovatio...,6 hours ago,NaN,NaN,NaN,NaN,NaN
i-ID-1,11162018,Canonical,canonical,Boston DCE,Python scripting experience. Data Center Engin...,https://www.indeed.com/rc/clk?jk=993a87e42e52d...,https://www.indeed.com/cmp/Canonical,Klaviyo,klaviyo,,...,NA,"Boston, MA","Klaviyo, Boston, MA",We love programming and the excitement that co...,7 hours ago,NaN,NaN,NaN,NaN,NaN
i-ID-2,11162018,Wavsys,wavsys,Roboticist,Deep theoretical foundation in relevant artifi...,https://www.indeed.com/company/Wavsys/jobs/Rob...,https://www.indeed.com/cmp/Wavsys,IBM,ibm,,...,https://d2q79iu7y748jz.cloudfront.net/s/_logo/...,"Littleton, MA 01460","IBM, Littleton, MA 01460",Job Description\nJob Description\nResponsibili...,6 hours ago,NaN,NaN,NaN,NaN,NaN
i-ID-3,11162018,"Circulation, Inc.",circulation,Technical Support Engineer,"With HTML/CSS, Javascript, Python, Web APIs, a...","https://www.indeed.com/company/Circulation,-In...","https://www.indeed.com/cmp/Circulation,-Inc.",Dassault Systèmes,dassault systèmes,,...,https://d2q79iu7y748jz.cloudfront.net/s/_squar...,"Waltham, MA","Dassault Systèmes, Waltham, MA","Exa part of the family of Simulia products, ba...",8 hours ago,NaN,NaN,NaN,NaN,NaN
i-ID-4,11162018,Agios Pharmaceuticals,agios pharmaceuticals,Bioinformaticist,"Strong hands-on programing skills, especially ...",https://www.indeed.com/company/Agios-Pharmaceu...,https://www.indeed.com/cmp/Agios-Pharmaceuticals,Continental Resources,continental resources,,...,NA,"Bedford, MA 01730","Continental Resources, Bedford, MA 01730",Outperform your competition; consider becoming...,6 hours ago,NaN,NaN,NaN,NaN,NaN
i-ID-5,11162018,Index Data,index data,DevOps Engineer,Intermediate knowledge of higher-level languag...,https://www.indeed.com/company/Index-Data/jobs...,https://www.indeed.com/cmp/Index-Data,IBM,ibm,,...,https://d2q79iu7y748jz.cloudfront.net/s/_logo/...,"Littleton, MA 01460","IBM, Littleton, MA 01460",Job Description\nAre you interested in pursuin...,6 hours ago,NaN,NaN,NaN,NaN,NaN
i-ID-6,11162018,Ace Technologies,ace technologies,Hadoop,Familiar with Hadoop tools/languages such as P...,https://www.indeed.com/rc/clk?jk=4cc6688f09b1e...,https://www.indeed.com/cmp/Ace-Technologies,Onduo,onduo,,...,NA,"Newton, MA 02458","Onduo, Newton, MA 02458",Responsibilities and Duties:\nOnduo is looking...,8 hours ago,NaN,NaN,NaN,NaN,NaN
i-ID-7,11162018,Positionskart,positionskart,Senior Business Analyst,"Development – Python, Powershell, Chef, VRO/VR...",https://www.indeed.com/company/Positionskart/j...,https://www.indeed.com/cmp/Positionskart,Klaviyo,klaviyo,,...,NA,"Boston, MA","Klaviyo, Boston, MA",We love programming and the excitement that co...,7 hours ago,NaN,NaN,NaN,NaN,NaN
i-ID-8,11162018,"Foundation Medicine, Inc.",foundation medicine,"Data Scientist, Clinicogenomics",Proficiency with statistical analysis software...,https://www.indeed.com/rc/clk?jk=7b542a241ad69...,https://www.indeed.com/cmp/Foundation-Medicine...,enEvolv,enevolv,,...,NA,"Medford, MA 02155","enEvolv, Medford, MA 02155",We are seeking a Microfluidics Engineer to joi...,6 hours ago,NaN,NaN,NaN,NaN,NaN


In [52]:
fetch.columns

Index(['pulldate', 'companies', 'l_companies', 'titles', 'description',
       'post_links', 'comp_links', 'company_name', 'l_company_name',
       'salaries_link', 'reviews_link', 'jobtitle', 'company_pic', 'location',
       'comp_loc', 'body', 'post_time', 'pulldates', 'locations', 'descr',
       'ezapply', 'company_pics'],
      dtype='object')

In [12]:
options = Options()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument('--no-sandbox')
options.add_argument("--disable-extensions")
options.add_argument('--log-level=3')
wd = webdriver.Chrome(scrap_params['chromedriver_location'], options=options)
url = 'https://www.indeed.com/jobs?q=python&l=worcester&start=0&sort=date&radius=25'
print(url)
wd.get(url)
companies = []

#companies = [company.text for company in wd.find_elements_by_xpath("//div[@data-tn-component='organicJob']")]
companies = [company.text for company in wd.find_elements_by_xpath("//div[@data-tn-component='organicJob']//span[@class='company']")]
titles = [title.text for title in wd.find_elements_by_xpath("//div[@data-tn-component='organicJob']//h2[@class='jobtitle']")]
description = [description.text for description in wd.find_elements_by_xpath("//div[@data-tn-component='organicJob']//span[@class='summary']")]
post_links = [post_link.get_attribute('href') for post_link in wd.find_elements_by_xpath("//div[@data-tn-component='organicJob']//h2[@class='jobtitle']//a")]
comp_links = ['https://www.indeed.com/cmp/' + str(company).replace(' ', '-') for company in companies]

tmp_df = pd.DataFrame({'ID' : 'ID', 'site' : 'indeed', 'title' : titles, 'company' : companies, 'summary' : description, 'post_links' : post_links, 'comp_links' : comp_links})

comp_list = []
for link in comp_links:
    wd.get(link)
    company_name = wd.find_element_by_xpath("//div[@class='cmp-company-name-container']").text
    try:
        average_comp_rating = wd.find_element_by_xpath("//span[@class='cmp-header-rating-average']").text
    except:
        average_comp_rating = 'Not ranked.'
    salaries_link = 'https://www.indeed.com/cmp/' + company_name + '/salaries/'
    reviews_link = 'https://www.indeed.com/cmp/' + company_name + '/reviews/'
    comp_list.append([company_name, average_comp_rating, salaries_link, reviews_link])
comp_df = pd.DataFrame(comp_list, columns=['company', 'rating', 'salaries_link', 'reviews_link'])

post_list = []
for link in post_links:
    wd.get(link)
    jobtitle = wd.find_element_by_xpath("//h3[contains(@class, 'jobsearch-JobInfoHeader-title')]").text
    #jobtitle = wd.find_element_by_xpath("//h3[@class='jobsearch-JobInfoHeader-title']").text
    #company_name = wd.find_element_by_xpath("//div[@class='icl-u-lg-mr--sm icl-u-xs-mr--xs']").text
    try:
        company_pic = wd.find_element_by_xpath("//img[@class='jobsearch-CompanyAvatar-image']").get_attribute('src')
    except:
        company_pic = 'NA'
    line = wd.find_element_by_xpath("//div[contains(@class, 'jobsearch-InlineCompanyRating')]").text
    location = line.split('\n')[2]
    company_name = line.split('\n')[0]
    body = wd.find_element_by_xpath("//div[contains(@class, 'jobsearch-JobComponent-description')]").text
    post_time = wd.find_element_by_xpath("//div[@class='jobsearch-JobMetadataFooter']").text.split('-')[1].strip()
    post_list.append([jobtitle, company_name, post_time, location, company_pic, body])
post_df = pd.DataFrame(post_list, columns=['title', 'company', 'posttime', 'location', 'company_pic', 'body_text'])
job_df = comp_df.merge(post_df, on='company', how='left')
job_df = job_df.merge(tmp_df, on=['company', 'title'], how='left')
wd.quit()

https://www.indeed.com/jobs?q=python&l=worcester&start=0&sort=date&radius=25
10 
 10 
 10 
 10 
 10
-----------------------------------------------------------------------------
-----------------------------------------------------------------------------
                        company       rating  \
0                  Diverse Lynx          4.8   
1                   PerkinElmer          3.8   
2                       Marvell          3.4   
3                       Marvell          3.4   
4                          Bose          4.2   
5                          Bose          4.2   
6                          Bose          4.2   
7                          Bose          4.2   
8                       Marvell          3.4   
9                       Marvell          3.4   
10                         Dell          4.0   
11                        Intel          4.1   
12  The Hanover Insurance Group          3.8   
13           Spill Center, Inc.  Not ranked.   

                       

In [8]:
tmp_df['comp_links']

0              https://www.indeed.com/cmp/Diverse-Lynx
1               https://www.indeed.com/cmp/PerkinElmer
2                   https://www.indeed.com/cmp/Marvell
3                      https://www.indeed.com/cmp/Bose
4                      https://www.indeed.com/cmp/Bose
5                   https://www.indeed.com/cmp/Marvell
6                     https://www.indeed.com/cmp/Intel
7                      https://www.indeed.com/cmp/DELL
8    https://www.indeed.com/cmp/The-Hanover-Insuran...
9        https://www.indeed.com/cmp/Spill-Center,-Inc.
Name: comp_links, dtype: object

In [152]:
def tex_output(fetch, suffix=''):
    fetch_df = fetch
    #fetch_dict = fetch[1]
    map_list = []
    content = r'''\UseRawInputEncoding%
\documentclass{{article}}%
\usepackage[T1]{{fontenc}}%
\usepackage[utf8]{{inputenc}}%
\inputencoding{utf8}
\usepackage{{lmodern}}%
\usepackage{graphicx}
\usepackage{{textcomp}}%
\usepackage{{lastpage}}%

\begin{document}%
\normalsize%'''
    
    for index, row in fetch_df.iterrows():
        print(index)
        print(row['map_url'])
        map_file_name = str(index) + '.jpg'
        map_list.append(map_file_name)
        urllib.request.urlretrieve(str(row['map_url']), map_file_name)
        content = content + r'''
\section{{{jobtitle}}}%
\subsection{{{comploc}}}
{{{address}}}
{{{distance}}}
{{{duration}}}


\begin{{figure}}
\includegraphics{{{map_file_name}}}
\end{{figure}}

\subsection{{Salary}}%
{{{salary}}}

\subsection{{Link}}%
{{{link}}}

\newpage


'''.format(**{'jobtitle': row['title'], 'salary' : row['salary'], 'comploc' : row['comp_loc'], 'address' : row['address'], 'distance' : row['distance'], 
              'duration' : row['duration'], 'link' : row['link'], 'map_file_name' : map_file_name}).replace('&', '\&').replace('$', '\$').replace('#', '\#').replace('|', '').replace('_', '\_')        
        filename = 'job_scrap_output' + suffix + '.tex'

    content = content + '''
\end{document}%'''
    with open(filename,'w', encoding='utf-8') as f:
        f.write(content)
    cmd = ['pdflatex', filename]
    subprocess.check_output(cmd)

    os.unlink('job_scrap_output.log')
    os.unlink('job_scrap_output.aux')
    for file in map_list:
        os.unlink(file)

In [18]:
def tex_output(fetch, suffix=''):
    fetch_df = fetch
    #fetch_dict = fetch[1]
    map_list = []
    content = r'''\documentclass[letterpaper,12pt,fleqn]{{article}}
\usepackage{{fancyhdr}}
\usepackage{{datetime}}
\usepackage{{hyperref}}

% Insert Title Page Info Below

\newcommand{\Titles}{Job ScrapyBoi} %
\newcommand{\cityset}{Boston} %
\newcommand{\jobset}{Data Science} %
\newcommand{\authors}{Matt and Chris} %
\newcommand{\sources}{Indeed, LinkedIn, StackOverflow} %
\newcommand{\lastwords}{End of Results} %

% Margins and Footer Style
\setlength{\topmargin}{0cm} %
\setlength{\textheight}{9.25in} %
\setlength{\oddsidemargin}{0.0in}
\setlength{\evensidemargin}{0.0in}
\setlength{\textwidth}{16cm}
\pagestyle{fancy}
\lhead{{}} 
\chead{{}} 
\rhead{{}} 
\lfoot{{}} 
\cfoot{{\footnotesize{{Page \thepage \ of \pageref{{finalpage}}}}}} 
\rfoot{{}} 

\renewcommand{\headrulewidth}{0pt} %Do not print a rule below the header
\renewcommand{\footrulewidth}{0pt}


\begin{document}

% Title page

\begin{center}
\vspace{10cm}
\huge\textbf{\Titles}
\end{center}
\vspace{4cm}

\begin{center}
\Large\textbf{\today}
\end{center}


\begin{center}
\Large\textbf{\cityset}
\end{center}

\begin{center}
\large\textbf{\jobset}
\end{center}
\vspace{4cm}

\begin{center}
\textit{Authors: \authors}
\end{center}

\begin{center}
\textit{Sources: \sources}
\end{center}
\newpage

% End title page'''
    
    for index, row in fetch_df.iterrows():
        print(index)
        #print(row['map_url'])
        #map_file_name = str(index) + '.jpg'
        #map_list.append(map_file_name)
        #urllib.request.urlretrieve(str(row['map_url']), map_file_name)
        tmp_content = r'''

\bigskip

\paragraph {%(jobtitle)s} Company

\noindent\rule{8cm}{0.4pt}
\begin{description}
\item \lbrack \ \textit{Map} ] %(map)s
\item \lbrack \ \textit{Distance} ] %(distance)s
\item \lbrack \ \textit{Drive Duration} ] %(duration)s
\item \lbrack \ \textit{Keywords} ] %(keywords)s
\item \lbrack \ \textit{Salary} ] %(salary)s
\item \lbrack \ \textit{Full Text} ] %(fulltext)s
\item \lbrack \ \textit{Link} ] \url{%(link)s}
\end{description}

\begin{center}
\vspace{3cm}
--------- \textit{\lastwords} ---------
\end{center}

\newpage

'''
        '''tmp_content = tmp_content.format(**)'''
        tmp_content = tmp_content % { 'jobtitle' : row['title'], 'map' : row['map_url'] , 'distance' : row['distance'], 'duration' : 'other butts' , 'keywords' : 'butty' , 'salary' : 'moneybutts', 
                                            'fulltext' : row['body_text'], 'link' : row['post_link'] }
        tmp_content = tmp_content.encode('ascii',errors='ignore').decode()
        tmp_content = tmp_content.replace('&', '\&').replace('$', '\$').replace('#', '\#').replace('|', '').replace('_', '\_')
        #tmp_content = codecs.encode(tmp_content, 'utf-8').decode('utf-8')
        content = content + tmp_content
        # ID 	title 	company 	location 	post_link 	comp_link 	ezapply 	company_pic 	pulldate 	comp_loc 	rating 	salaries_link 	reviews_link 	l_company 	postdate 	body_text 	summary 	benefits
        filename = 'job_scrap_output' + suffix + '.tex'

    content = content + '''\end{document}'''
    with open(filename,'w', encoding='utf-8') as f:
        f.write(content)
    cmd = ['pdflatex', filename]
    subprocess.check_output(cmd)

    os.unlink('job_scrap_output.log')
    os.unlink('job_scrap_output.aux')
    #for file in map_list:
        #os.unlink(file)

In [19]:
tex_output(fetch)

L-ID-0
L-ID-1
L-ID-2
L-ID-3
L-ID-4
L-ID-5
L-ID-6
L-ID-7
L-ID-8
L-ID-9
L-ID-10
L-ID-11
L-ID-12
L-ID-13
L-ID-14
L-ID-15
L-ID-16
L-ID-17
L-ID-18
L-ID-19
L-ID-20
L-ID-21
L-ID-22
L-ID-23
L-ID-24
i-ID-0
i-ID-1
i-ID-2
i-ID-3
i-ID-4
i-ID-5
i-ID-6
i-ID-7
i-ID-8
i-ID-9
i-ID-10
i-ID-11
i-ID-12
i-ID-13
i-ID-14
i-ID-15
i-ID-16
i-ID-17
i-ID-18
i-ID-19
i-ID-20
i-ID-21
i-ID-22
i-ID-23
i-ID-24


In [ ]:
### Here's below

'''indeed_pg_num = 0
                site_ind = 0 # set to 0 at start of site and compared to results so there are 'results' number of pulls per site
                list_o_divs = [] # holds divs if multiple pages
                tmp_dict = {} # holds info on each div loop
                # https://www.indeed.com/jobs?q=data+science&l=boston&sort=date
                res_left = results
                base = 'https://www.indeed.com/jobs?'
                while res_left > 0:            
                    params = {'q' : job_title,'l' : city, 'start' : (results-res_left),  'sort' : 'date', 'radius': '25'}
                    page = requests.get(base, params) 
                    time.sleep(1)  
                    soup = get_soup(page.text)
                    divs = soup.find_all(name="div", attrs={"class":"row"})
                    list_o_divs.append(divs)
                    res_left = res_left - len(divs)
                    indeed_pg_num = indeed_pg_num + 1
                print('total indeed pages : ' + str(indeed_pg_num))
                for divs in list_o_divs:
                    for div in divs:
                        if (site_ind < results):
                            tmp_dict['sID'] = 'ID' + str(num)
                            tmp_dict['site'] = site
                            tmp_dict['pulldate'] = str(time.strftime("%M%D%y", time.localtime(time.time())))
                            tmp_dict['link'] = 'http://www.indeed.com' + div.find(name='a', attrs={'class':'turnstileLink'})['href']
                            #tmp_dict['link'] = extract_link(div, site)
                            tmp_dict['title'] = div.find(name='a', attrs={'class':'turnstileLink'})['title']
                            # tmp_dict['title'] = extract_job_title(div, site)
                            tmp_dict['summary'] = div.find('span', attrs={'class': 'summary'}).text.strip()
                            # tmp_dict['summary'] = extract_summary(div, site)
                            #tmp_dict['salary'] = extract_salary(div, site)
                            # tmp_dict['location'] = extract_location(div, site).strip() # two things
                            if div.find('span', attrs={'class': 'location'}).text is None:
                                tmp_dict['location'] = div.find('div', attrs={'class': 'location'}).text.strip()
                            else:
                                tmp_dict['location'] = div.find('span', attrs={'class': 'location'}).text.strip()
                            tmp_dict['tags'] = extract_tags(div, site)
                            #tmp_dict['id'] = extract_id(div, site)
                            tmp_dict['id'] = div['id']
                            if div.find('div', attrs={'class': 'iaP'}) is not None:
                                tmp_dict['ezapply'] = True
                            else:
                                tmp_dict['ezapply'] = False
                            # tmp_dict['ezapply'] = extract_easyapply(div, site)
                            tmp_dict['postdate'] = extract_postdate(div, site)
                            #tmp_dict['company'] = extract_company(div, site).strip()
                            tmp_dict['company'] = div.find(name="span", attrs={"class":"company"}).text.replace('\n', '').strip()
                            tmp_dict['comp_loc'] = str(extract_company(div, site).strip() + ', ' + extract_location(div, site).strip())
                            tmp_df = pd.DataFrame(tmp_dict, index=[tmp_dict['sID']])
                            job_df = job_df.append(tmp_df, sort=False)
                            num = num + 1
                            site_ind = site_ind + 1'''

In [ ]:
'''                site_ind = 0 # set to 0 at start of site and compared to results so there are 'results' number of pulls per site
                list_o_divs = [] # holds divs if multiple pages
                tmp_dict = {} # holds info on each div loop
                # https://stackoverflow.com/jobs?q=data+scientist&l=boston&d=20&u=Miles
                pg = 1 # page counter 
                base = 'https://stackoverflow.com/jobs?'
                tot_pg = ((results // 20) + 1)
                print('Stack overflow total pages: ' + str(tot_pg))
                while pg <= tot_pg: 
                    params = {'q' : job_title, 'sort' : 'p', 'l' : city, 'd' : '20', 'u': 'Miles', 'pg': pg} 
                    page = requests.get(base, params) 
                    print(page.url)
                    time.sleep(1)
                    soup = get_soup(page.text)
                    divs = soup.find_all(name="div", attrs={"class":"-job-summary"})
                    list_o_divs.append(divs)
                    pg = pg + 1
                for divs in list_o_divs:
                    for div in divs:
                        if (site_ind < results): # if there is a div past the number of results for the site, doesn't add it to tmp_dict
                            tmp_dict['sID'] = 'ID' + str(num)
                            tmp_dict['site'] = site
                            tmp_dict['pulldate'] = str(time.strftime("%m-%d-%Y_%I-%M_%p", time.localtime(time.time())))
                            tmp_dict['link'] = 'https://www.stackoverflow.com' + div.find(name='a', attrs={'class':'s-link'})['href']
                            # tmp_dict['link'] = extract_link(div, site)
                            tmp_dict['title'] = div.find(name='a', attrs={'class':'s-link'})['title']
                            # tmp_dict['title'] = extract_job_title(div, site)
                            tmp_dict['summary'] = 'No Stack Summary yet'
                            if div.find('div', attrs={'class': '-perks'}).text is None:
                                tmp_dict['salary'] = 'NA'
                            else:
                                div.find('div', attrs={'class': '-perks'}).text.replace('\n', ' ').replace('\r', '').strip()
                            # tmp_dict['salary'] = extract_salary(div, site)
                            tmp_dict['location'] = div.find_all('div', attrs={'class': '-company'}).split('-')[1].text
                            # tmp_dict['location'] = extract_location(div, site).strip()
                            tmp_dict['tags'] = ', '.join(div.find_all('a', attrs={'class': 'post-tag job-link no-tag-menu'}))
                            # tmp_dict['tags'] = extract_tags(div, site)
                            tmp_dict['id'] = div.find('span', attrs={'class': 'fav-toggle'})['data-jobid']
                            #tmp_dict['id'] = extract_id(div, site)
                            tmp_dict['ezapply'] = 'No Easy Applications on Stack'
                            if div.find('span', attrs={'class': 'ps-absolute pt2 r0 fc-black-500 fs-body1 pr12 t24'}).text is None:
                                tmp_dict['postdate'] = div.find('span', attrs={'class': 'ps-absolute pt2 r0 fc-black-500 fs-body1 pr12 t32'}).text
                            else:
                                tmp_dict['postdate'] = div.find('span', attrs={'class': 'ps-absolute pt2 r0 fc-black-500 fs-body1 pr12 t24'}).text
                            # tmp_dict['postdate'] = extract_postdate(div, site)
                            tmp_dict['company'] = div.find_all(name="div", attrs={"class":"-company"}).split('-')[0].text.replace('\n', '').replace('\r','')
                            #tmp_dict['company'] = extract_company(div, site).split('-')[0].strip()
                            tmp_dict['comp_loc'] = div.find_all('div', attrs={'class': '-company'}).text
                            tmp_df = pd.DataFrame(tmp_dict, index=[tmp_dict['sID']])
                            job_df = job_df.append(tmp_df, sort=False)
                            num = num + 1
                            site_ind = site_ind + 1'''